# Inicializace

In [158]:
from collections import deque               # Ukladani stavu
from __future__ import division             # Deleni realnych cisel (kvuli verzi Pythonu 2.6)
import sys                                  # Pro navazani knihovny Open AI Gym
import numpy as np                          # Knihovna pro operace s maticemi
sys.path.append('/home/xbucha02/libraries') # Adresa knihovny Open AI Gym
import gym                                  # Knihovna Open AI Gym
from gym import wrappers                    # Pomucka na nahravani
env = gym.make('CartPole-v0')            # Konkretni hra z Open AI Gym
env = wrappers.Monitor(env, '/home/lachubcz/tmp/cartpole-experiment-1', force=True) #Nahravani zaznamu
gpuMemoryUsage=1                            # Vyuziti pameti GPU
import tensorflow as tf                     # Knihovna TensorFlow pro sdileni GPU
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = gpuMemoryUsage
set_session(tf.Session(config=config))
import random

# Parametry
observetime = 500                  # Pocet framu kazde hry
startEpsilon = 1                   # Pravdepodobnost konani nahodneho tahu na zacatku
endEpsilon = 0.1                   # Pravdepodobnost konani nahodneho tahu na konci
currentEpsilon = startEpsilon      # Soucasna pravdepodobnost konani nahodneho tahu
epsilonDiminution = 0.995          # Hodnota snizovani epsilonu
gamma = 0.95                       # Snizujici faktor
minibatchSize = 32                 # Velikost minibatche
episodes = 5000                    # Pocet epizod
#numberOfGames = 20                # Pocet her na konci
actionCount = env.action_space.n   # Pocet akci v danem prostredi
stateSize = env.observation_space.shape[0]
learningRate = 0.001             # Learning rate
memorySize = 2000                  # Replay pamet

# Model NN

In [159]:
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics

def getDQN(actionCount):
    net = Sequential()
    #net.add(Dense(20, activation="relu", kernel_initializer="uniform", input_shape=(2, 2)))
    #net.add(Flatten())
    net.add(Dense(24, activation="relu", input_shape=(2,4)))
    #net.add(Flatten())
    net.add(Dense(24, activation="relu"))
    #net.add(Dropout(rate=0.5))
    #net.add(Dense(8, activation="relu", kernel_initializer="uniform"))
    
    net.add(Dense(actionCount, activation="linear"))
    
    net.summary()
    net.compile(loss=losses.mean_squared_error, optimizer=optimizers.Adam(lr=learningRate), metrics=['accuracy'])
    
    return net

In [160]:
net = getDQN(actionCount)
observation = env.reset()                    # Reset prostredi
memory = deque(maxlen=memorySize)

for eps in range (episodes):  
    print("{}:lol:{}" .format(state, state[0][1]))#np.reshape(state, [1, stateSize])
    #D = deque() #vyprazdneni D
    env.reset()
    env.render()
    #obs = np.expand_dims(observation, axis=0) 
    #state = np.stack((obs, obs), axis=1)

    i = 0
    for t in range(observetime):
        #if i == 0:
            
        #else:
        #    if i==4:
        #        i = 0
        #    else:
        #        i = i + 1
        if np.random.rand() <= currentEpsilon:
            action = np.random.randint(0, actionCount, size=1)[0]
        else:
            #print("shirt")
            Q = net.predict(state)
            #print("shirt")
            action = np.argmax(Q)
            #print("{}".format(action))
        #print ("state: {}" .format(state))
        #print("{}".format(action))
        observation_new, reward, done, info = env.step(int(action))
        
        #obs_new = np.expand_dims(observation_new, axis=0)        
        #print ("obs_new: {}" .format(obs_new))
        #state_new = np.append(np.expand_dims(obs_new, axis=0), state[:, :1, :], axis=1)
        #state = state[0][0]#next_state = np.reshape(next_state, [1, stateSize])
        memory.append((state, action, reward, state_new, done))
        #print()
        print ("state: {} state_new: {}" .format(state, state_new))
        
        state = state_new         # Update state
        #print ("state: {}" .format(state))
        #print ("__________________________________________")
        if done:
            print("Episode: {}/{}, score: {}".format(eps, episodes, t))
            break
    if eps > 100:
        minibatch = random.sample(memory, minibatchSize) #z D vybere pocet mb_size samplu

        #inputs_shape = (minibatchSize,) + state.shape[1:]
        #print("inputs_shape: {}".format(inputs_shape))
        #inputs = np.zeros(inputs_shape)
        #print("inputs: {}".format(inputs))
        #targets = np.zeros((minibatchSize, env.action_space.n)) #vytvori pole ([], []), 1. argument - radky, 2. argument - sloupce
        #print("targets: {}".format(targets))

        for i in range(0, minibatchSize):
            stateMb = minibatch[i][0]
            actionMb = minibatch[i][1]
            rewardMb = minibatch[i][2]
            state_newMb = minibatch[i][3]
            doneMb = minibatch[i][4]

            #target = reward

            # Build Bellman equation for the Q function
            #inputs[i:i+1] = np.expand_dims(state, axis=0)

            #print("inputs: {}".format(inputs))
            #targets[i] = net.predict(state)
            #print("targets: {}".format(targets))
            Q_sa = net.predict(state_newMb)

            if done:
                targetMb = rewardMb
            else:
                targetMb = reward + gamma * np.max(Q_sa)

            target_f = net.predict(stateMb)
            target_f[0][action] = targetMb

            #print("inputs: {}, targets:{}" .format(inputs, targets))

            net.fit(stateMb, target_f, epochs=1, verbose=0)

        #net.train_on_batch(inputs, targets)

        if currentEpsilon > endEpsilon:
            currentEpsilon = currentEpsilon * epsilonDiminution
        print("{}".format(currentEpsilon))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_147 (Dense)            (None, 2, 24)             120       
_________________________________________________________________
dense_148 (Dense)            (None, 2, 24)             600       
_________________________________________________________________
dense_149 (Dense)            (None, 2, 2)              50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________
[[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]:lol:[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422

  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
Episode: 6/5000, score: 12
[[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]:lol:[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.1101

  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.0

  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.0

state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
Episode: 24/5000, score: 23
[[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]


...

state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.203

  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
Episode: 84/5000, score: 23
[[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]:lol:[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.110

  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.0

  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
Episode: 101/5000, score: 10
0.995
[[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]:lol:[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]] state_new: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  [-0.11015386 -1.20338865  0.18124085  1.86504052]]]
state: [[[-0.13422163 -1.01065592  0.21854166  1.6336702 ]
  

AssertionError: 3 (<type 'int'>) invalid